In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_data = "/content/train_data.txt"
data = pd.read_csv(train_data, sep=':::', names=['Title','genre','description'])

<ipython-input-2-f4d7027dff8d>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(train_data, sep=':::', names=['Title','genre','description'])


In [ ]:
# Preprocess the data
X = data['description']
y = data['genre']

In [ ]:
# Label encoding for genres
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Vectorize the text data
max_words = 1000  # Adjust according to your dataset and computing power
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 100  # Adjust according to the maximum length of your descriptions
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [ ]:
print(len(X_train_pad[0]))

100


In [ ]:
print

In [ ]:
# ANN model
ann_model = Sequential([
    Dense(64, activation='relu', input_shape=(max_len,)),
    Dense(32, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])
ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN model
ann_model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

Epoch 1/10
681/681 [==============================] - 3s 3ms/step - loss: 10.4096 - accuracy: 0.2251 - val_loss: 2.8031 - val_accuracy: 0.2462
Epoch 2/10
681/681 [==============================] - 2s 3ms/step - loss: 2.6248 - accuracy: 0.2495 - val_loss: 2.5151 - val_accuracy: 0.2475
Epoch 3/10
681/681 [==============================] - 2s 3ms/step - loss: 2.4338 - accuracy: 0.2499 - val_loss: 2.4148 - val_accuracy: 0.2475
Epoch 4/10
681/681 [==============================] - 1s 2ms/step - loss: 2.3689 - accuracy: 0.2500 - val_loss: 2.3817 - val_accuracy: 0.2475
Epoch 5/10
681/681 [==============================] - 1s 2ms/step - loss: 2.3487 - accuracy: 0.2501 - val_loss: 2.3706 - val_accuracy: 0.2475
Epoch 6/10
681/681 [==============================] - 2s 2ms/step - loss: 2.3417 - accuracy: 0.2503 - val_loss: 2.3663 - val_accuracy: 0.2481
Epoch 7/10
681/681 [==============================] - 2s 2ms/step - loss: 2.3387 - accuracy: 0.2502 - val_loss: 2.3585 - val_accuracy: 0.2479
Epoch

In [ ]:
# RNN model
rnn_model = Sequential([
    Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    LSTM(64),
    Dense(len(label_encoder.classes_), activation='softmax')
])
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the RNN model
rnn_model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

Epoch 1/10
681/681 [==============================] - 51s 71ms/step - loss: 2.2420 - accuracy: 0.3515 - val_loss: 2.0576 - val_accuracy: 0.4179
Epoch 2/10
681/681 [==============================] - 46s 67ms/step - loss: 1.9520 - accuracy: 0.4492 - val_loss: 1.9463 - val_accuracy: 0.4374
Epoch 3/10
681/681 [==============================] - 48s 71ms/step - loss: 1.8379 - accuracy: 0.4718 - val_loss: 1.8872 - val_accuracy: 0.4504
Epoch 4/10
681/681 [==============================] - 47s 69ms/step - loss: 1.7418 - accuracy: 0.4970 - val_loss: 1.8219 - val_accuracy: 0.4668
Epoch 5/10
681/681 [==============================] - 48s 71ms/step - loss: 1.6586 - accuracy: 0.5177 - val_loss: 1.7679 - val_accuracy: 0.4846
Epoch 6/10
681/681 [==============================] - 50s 74ms/step - loss: 1.5791 - accuracy: 0.5417 - val_loss: 1.7545 - val_accuracy: 0.4853
Epoch 7/10
681/681 [==============================] - 49s 71ms/step - loss: 1.5178 - accuracy: 0.5564 - val_loss: 1.7241 - val_accuracy:

In [ ]:
# KNN model
# Convert text data to bag-of-words representation
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_vec, y_train)

KNeighborsClassifier()

In [ ]:
# Evaluate models
ann_pred = np.argmax(ann_model.predict(X_test_pad), axis=-1)
rnn_pred = np.argmax(rnn_model.predict(X_test_pad), axis=-1)
knn_pred = knn_model.predict(X_test_vec)

171/171 [==============================] - 3s 16ms/step


In [ ]:
print("ANN Accuracy:", accuracy_score(y_test, ann_pred))
print("RNN Accuracy:", accuracy_score(y_test, rnn_pred))
print("KNN Accuracy:", accuracy_score(y_test, knn_pred))

ANN Accuracy: 0.2480719794344473
RNN Accuracy: 0.4944913698127066
KNN Accuracy: 0.3235402130003672
